In [1]:
%pip install tensorflow -q
%pip install pandas -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from tensorflow.keras import models, layers, activations,\
optimizers, utils, losses, initializers, metrics, callbacks

In [3]:
epochs = 10000
batch_size = 120
patience = 48
learning_rate = 1e-4
model_path = 'checkpoints/model.keras'
train_path = 'train'
exists = os.path.exists(model_path)
items = len(os.listdir(train_path))
classes = 1 if items < 3 else items
print(classes)

17


In [4]:
# Carrega modelo se já existir um checkpoint, caso contrário, o cria.
model = models.load_model(model_path) \
if exists \
else models.Sequential([
  layers.Resizing(120, 120),
  layers.Rescaling(1.0/255),
  layers.RandomRotation((-0.1, 0.1)),
  layers.RandomFlip(),
  layers.BatchNormalization(),
  
  layers.Conv2D(40, (7, 7),
    activation = 'relu',
    kernel_initializer = initializers.RandomUniform()
  ),
  layers.MaxPooling2D((2, 2)),
  
  layers.Conv2D(60, (5, 5),
    activation = 'relu',
    kernel_initializer = initializers.RandomUniform()
  ),
  layers.MaxPooling2D((2, 2)),
  
  layers.Conv2D(80, (3, 3),
    activation = 'relu',
    kernel_initializer = initializers.RandomUniform()
  ),
  layers.MaxPooling2D((2, 2)),
  
  layers.Flatten(),
  
  layers.Dropout(0.5),
  layers.Dense(102,
    activation = 'relu',
    kernel_initializer = initializers.RandomUniform()
  ),
  
  layers.Dropout(0.2),
  layers.Dense(100,
    activation = 'relu',
    kernel_initializer = initializers.RandomUniform()
  ),
  
  layers.Dropout(0.2),
  layers.Dense(102,
    activation = 'relu',
    kernel_initializer = initializers.RandomUniform()
  ),
  
  layers.Dropout(0.2),
  layers.Dense(100,
    activation = 'relu',
    kernel_initializer = initializers.RandomUniform()
  ),
  
  layers.Dropout(0.2),
  layers.Dense(102,
    activation = 'relu',
    kernel_initializer = initializers.RandomUniform()
  ),
  
  layers.Dropout(0.2),
  layers.Dense(100,
    activation = 'relu',
    kernel_initializer = initializers.RandomUniform()
  ),
  
  layers.Dropout(0.2),
  layers.Dense(68,
    activation = 'relu',
    kernel_initializer = initializers.RandomUniform()
  ),
  
  layers.Dense(classes,
    activation = 'sigmoid',
    kernel_initializer = initializers.RandomUniform()
  )
])
print(exists)
if not exists:
  model.compile(
    optimizer = optimizers.Adam(
      learning_rate = learning_rate
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [ 'accuracy' ]
  )
else:
  model.summary()
  
train = utils.image_dataset_from_directory(
  train_path,
  validation_split= 0.2,
  subset= "training",
  seed= 123,
  shuffle= True,
  image_size= (224, 224),
  batch_size= batch_size
)

test = utils.image_dataset_from_directory(
  train_path,
  validation_split= 0.2,
  subset= "validation",
  seed= 123,
  shuffle= True,
  image_size= (224, 224),
  batch_size= batch_size
)



True
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 120, 120, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 120, 120, 3)       0         
                                                                 
 random_rotation (RandomRot  (None, 120, 120, 3)       0         
 ation)                                                          
                                                                 
 random_flip (RandomFlip)    (None, 120, 120, 3)       0         
                                                                 
 batch_normalization (Batch  (None, 120, 120, 3)       12        
 Normalization)                                                  
                                                                 
 conv2d (Conv2D)             (None, 114, 114, 40)

In [5]:
lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9)

model.fit(train,
  epochs = epochs,
  validation_data = test,
  callbacks= [
    callbacks.EarlyStopping(
      monitor = 'val_loss',
      patience = patience,
      verbose = 1
    ),
    callbacks.ModelCheckpoint(
      filepath = model_path,
      save_weights_only = False,
      monitor = 'loss',
      mode = 'min',
      save_best_only = True
    ),
    callbacks.LearningRateScheduler(
      lr_schedule 
    )
  ]
)

Epoch 1/10000




12/12 [==============================] - 12s 818ms/step - loss: 2.8276 - accuracy: 0.1097 - val_loss: 2.8035 - val_accuracy: 0.1167 - lr: 0.0010
Epoch 2/10000
12/12 [==============================] - 11s 863ms/step - loss: 2.8054 - accuracy: 0.1097 - val_loss: 2.8009 - val_accuracy: 0.1167 - lr: 9.9999e-04
Epoch 3/10000
12/12 [==============================] - 11s 861ms/step - loss: 2.8205 - accuracy: 0.1111 - val_loss: 2.8257 - val_accuracy: 0.1194 - lr: 9.9998e-04
Epoch 4/10000
12/12 [==============================] - 11s 856ms/step - loss: 2.8047 - accuracy: 0.1118 - val_loss: 2.7884 - val_accuracy: 0.1333 - lr: 9.9997e-04
Epoch 5/10000
12/12 [==============================] - 11s 871ms/step - loss: 2.7693 - accuracy: 0.1257 - val_loss: 2.7095 - val_accuracy: 0.1361 - lr: 9.9996e-04
Epoch 6/10000
12/12 [==============================] - 11s 858ms/step - loss: 2.7186 - accuracy: 0.1361 - val_loss: 2.7232 - val_accuracy: 0.1417 - lr: 9.9995e-04
Epoch 7/10000
12/12 [=================